In [12]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser, CommaSeparatedListOutputParser, StrOutputParser
import boto3
import os
import re
import json
from datetime import datetime
from pydantic import BaseModel
import importlib
import shutil
from utils import utils
import pandas as pd
from utils.time_converter import TimeConverter
importlib.reload(utils)
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")

In [13]:
DEEPSEEK_API_KEY

'sk-602a9dbe731d48b0afab59d3c140c583'

In [57]:
subject = '수학'
room_id = '674e58163925f28f6caf4fa0'

file_keys = utils.get_items('pagecall-text', f'{subject}/{room_id}')
utils.download_items('pagecall-text', file_keys, './downloads')

In [58]:
raw_data = utils.merge_files('./downloads')
shutil.rmtree('./downloads')

In [59]:
teacher_extracted_data = utils.extract_speaker(raw_data, speaker='teacher')
teacher_splited_data = utils.split_sentences(teacher_extracted_data)
student_extracted_data = utils.extract_speaker(raw_data, speaker='student')
student_splited_data = utils.split_sentences(student_extracted_data)

In [60]:
teacher_splited_data = utils.mapping_time(teacher_extracted_data, teacher_splited_data)
teacher_splited_data

[{'idx': 0, 'text': '목소리 잘 들릴까요?', 'start': '4m 54.5s', 'end': '4m 58.3s'},
 {'idx': 1, 'text': '안녕하세요.', 'start': '4m 58.5s', 'end': '5m 0.6s'},
 {'idx': 2,
  'text': '잘 들리고 그러면 필기 한번 확인해 볼게요.',
  'start': '5m 0.7s',
  'end': '5m 6.2s'},
 {'idx': 3, 'text': '잘 보여요?', 'start': '5m 9.7s', 'end': '5m 12.5s'},
 {'idx': 4, 'text': '잠깐만요.', 'start': '5m 12.5s', 'end': '5m 12.6s'},
 {'idx': 5,
  'text': '혹시 전에 선택으로 과외 해본 적 있어요?',
  'start': '5m 23.3s',
  'end': '5m 35.5s'},
 {'idx': 6, 'text': '아 정말요?', 'start': '5m 35.5s', 'end': '5m 38.1s'},
 {'idx': 7,
  'text': '저 되게 첫 수업이라서 조금 떨리는데 이거 화면 바뀔 때 같이 바뀌나요?',
  'start': '5m 38.8s',
  'end': '5m 57.2s'},
 {'idx': 8,
  'text': '이거 레벨테스트 봤는데 이거 두 개를 풀어줬더라구요',
  'start': '5m 59.9s',
  'end': '6m 23.0s'},
 {'idx': 9, 'text': '아 그거 괜찮아요', 'start': '5m 59.9s', 'end': '6m 23.0s'},
 {'idx': 10,
  'text': '같이 하면 되니까 이게 왜 두 개였냐면 이게 상반기용이 하나 있었고, 하반기용이 하나 있었거든요.',
  'start': '5m 59.9s',
  'end': '6m 33.3s'},
 {'idx': 11,
  'text': '그래서 이 파트가 지금 하반기용이고, 이

In [61]:
student_splited_data = utils.mapping_time(student_extracted_data, student_splited_data)
student_splited_data

[{'idx': 0, 'text': '감사합니다.', 'start': '0m 0.0s', 'end': '0m 2.4s'},
 {'idx': 1, 'text': '네 안녕하세요', 'start': '4m 41.8s', 'end': '5m 10.4s'},
 {'idx': 2, 'text': '네 네', 'start': '4m 41.8s', 'end': '5m 10.4s'},
 {'idx': 3, 'text': '네, 같이 볼게요.', 'start': '5m 35.0s', 'end': '5m 55.5s'},
 {'idx': 4,
  'text': '어떤 걸 보여줄지 모르겠어가지고..',
  'start': '6m 4.0s',
  'end': '6m 33.1s'},
 {'idx': 5, 'text': '아니 근데..', 'start': '6m 4.0s', 'end': '6m 33.1s'},
 {'idx': 6,
  'text': '근데 솔직히 볼 수 있는 게 몇 개 없어가지고..',
  'start': '6m 4.0s',
  'end': '6m 33.1s'},
 {'idx': 7,
  'text': '거의.. 아니 그니까 후툰 게 몇 개 있긴 한데요..',
  'start': '6m 4.0s',
  'end': '6m 33.1s'},
 {'idx': 8, 'text': '그..', 'start': '6m 4.0s', 'end': '6m 33.1s'},
 {'idx': 9, 'text': '모르는 게 너무 많아가지고..', 'start': '6m 4.0s', 'end': '6m 33.1s'},
 {'idx': 10, 'text': '네..', 'start': '6m 4.0s', 'end': '6m 33.1s'},
 {'idx': 11,
  'text': '아니 진짜 충격부모일 수도 있어요.',
  'start': '6m 4.0s',
  'end': '6m 33.1s'},
 {'idx': 12, 'text': '네..', 'start': '6m 33.2s', 'end':

In [62]:
teacher_df = pd.DataFrame(teacher_splited_data).rename(columns={"idx": "teacher_idx", "text": "teacher_text"})
student_df = pd.DataFrame(student_splited_data).rename(columns={"idx": "student_idx", "text": "student_text"})

teacher_df['start_ms'] = teacher_df['start'].apply(lambda x: TimeConverter.convert_time_to_milliseconds(x))
student_df['start_ms'] = student_df['start'].apply(lambda x: TimeConverter.convert_time_to_milliseconds(x))

df = pd.concat([teacher_df, student_df], ignore_index=True)
df = df.sort_values(by=["start_ms", "teacher_idx", "student_idx"]).reset_index(drop=True).drop(columns=["start_ms"])
df = df.astype({'teacher_idx': 'Int64', 'student_idx': 'Int64'})
df['time'] = df['start'] + " ~ " + df['end']
df = df.drop(columns=['start', 'end'])
df = df[['teacher_idx', 'student_idx', 'time', 'teacher_text', 'student_text']]

teacher_df = df[df['teacher_text'].notnull()].drop(columns=['student_text', 'student_idx', 'time']).rename(columns={"teacher_idx": "idx", "teacher_text": "text"}).reset_index(drop=True)
student_df = df[df['student_text'].notnull()].drop(columns=['teacher_text', 'teacher_idx', 'time']).rename(columns={"student_idx": "idx", "student_text": "text"}).reset_index(drop=True)
df

,teacher_idx,student_idx,time,teacher_text,student_text
0,<NA>,0,0m 0.0s ~ 0m 2.4s,NaN,감사합니다.
1,<NA>,1,4m 41.8s ~ 5m 10.4s,NaN,네 안녕하세요
2,<NA>,2,4m 41.8s ~ 5m 10.4s,NaN,네 네
3,0,<NA>,4m 54.5s ~ 4m 58.3s,목소리 잘 들릴까요?,NaN
4,1,<NA>,4m 58.5s ~ 5m 0.6s,안녕하세요.,NaN
...,...,...,...,...,...
1377,<NA>,440,117m 52.3s ~ 117m 52.6s,NaN,안녕히 계세요.
1378,937,<NA>,118m 7.7s ~ 118m 8.4s,안녕하세요.,NaN
1379,938,<NA>,118m 8.5s ~ 118m 11.0s,처음인데 미안해요.,NaN
1380,939,<NA>,118m 11.1s ~ 118m 11.9s,미안하네요.,NaN


In [110]:
chunks_with_overlap = utils.split_with_overlap(student_df, chunk_size=30, overlap=5)
chunks_with_overlap

[[{'idx': 0, 'text': '감사합니다.'},
  {'idx': 1, 'text': '네 안녕하세요'},
  {'idx': 2, 'text': '네 네'},
  {'idx': 3, 'text': '네, 같이 볼게요.'},
  {'idx': 4, 'text': '어떤 걸 보여줄지 모르겠어가지고..'},
  {'idx': 5, 'text': '아니 근데..'},
  {'idx': 6, 'text': '근데 솔직히 볼 수 있는 게 몇 개 없어가지고..'},
  {'idx': 7, 'text': '거의.. 아니 그니까 후툰 게 몇 개 있긴 한데요..'},
  {'idx': 8, 'text': '그..'},
  {'idx': 9, 'text': '모르는 게 너무 많아가지고..'},
  {'idx': 10, 'text': '네..'},
  {'idx': 11, 'text': '아니 진짜 충격부모일 수도 있어요.'},
  {'idx': 12, 'text': '네..'},
  {'idx': 13, 'text': '그런가요?'},
  {'idx': 14, 'text': '네.'},
  {'idx': 15, 'text': '네.'},
  {'idx': 16, 'text': '네.'},
  {'idx': 17, 'text': '네.'},
  {'idx': 18, 'text': '괜찮아요.'},
  {'idx': 19, 'text': '음 음 음'},
  {'idx': 20, 'text': '응.'},
  {'idx': 21, 'text': '응.'},
  {'idx': 22, 'text': '네.'},
  {'idx': 23, 'text': '아.'},
  {'idx': 24, 'text': '아.'},
  {'idx': 25, 'text': '오오'},
  {'idx': 26, 'text': '허성고요?'},
  {'idx': 27, 'text': '몰라요.'},
  {'idx': 28, 'text': '네 네 네 네 네'},
  {'idx': 29, 'text': 

In [111]:
from prompt import question_analyzer

# llm = ChatOpenAI(model='gpt-4o')
llm = ChatOpenAI(
    model='deepseek-chat', 
    openai_api_key=DEEPSEEK_API_KEY, 
    openai_api_base='https://api.deepseek.com',
)

importlib.reload(question_analyzer)
system_prompt = question_analyzer.QuestionAnalyzer(subject, user='학생').prompt
prompt = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    ('user', "{user_message}")
])
chain = prompt | llm | StrOutputParser()

# Run it in parallel over chunks
question_analysis_results = chain.batch([
    {"user_message": chunk} for chunk in chunks_with_overlap
])

In [112]:
def get_question_context_v2(df, target_indices, range_size=5):
    grouped_result = []
    valid_data  = df[df['student_idx'].notna()].reset_index()
    idx_mapping = valid_data.set_index('student_idx')['index'].to_dict()
    question_mapping = valid_data.set_index('student_idx')['student_text'].to_dict()
   
    for idx in target_indices:
        if idx not in idx_mapping:
            continue
        start_idx = max(0, idx_mapping[idx] - range_size)
        end_idx = min(len(df), idx_mapping[idx] + range_size + 1)
        contexts = df[start_idx:end_idx].to_dict(orient='records')
        
        context = []
        for item in contexts:
            if item['teacher_idx'] != None:
                context.append({'time': item['time'], 'teacher_text': item['teacher_text']})
            else:
                context.append({'time': item['time'], 'student_text': item['student_text']})
        
        grouped_result.append({'idx': idx, 'question': question_mapping[idx], 'context': context})
    return grouped_result


In [174]:
question_indices = utils.extract_question_indices(question_analysis_results)
question_indices
question_context = get_question_context_v2(df, question_indices)
question_context

[{'idx': 4,
  'question': '어떤 걸 보여줄지 모르겠어가지고..',
  'context': [{'time': '5m 35.5s ~ 5m 38.1s', 'teacher_text': '아 정말요?'},
   {'time': '5m 38.8s ~ 5m 57.2s',
    'teacher_text': '저 되게 첫 수업이라서 조금 떨리는데 이거 화면 바뀔 때 같이 바뀌나요?'},
   {'time': '5m 59.9s ~ 6m 23.0s',
    'teacher_text': '이거 레벨테스트 봤는데 이거 두 개를 풀어줬더라구요'},
   {'time': '5m 59.9s ~ 6m 23.0s', 'teacher_text': '아 그거 괜찮아요'},
   {'time': '5m 59.9s ~ 6m 33.3s',
    'teacher_text': '같이 하면 되니까 이게 왜 두 개였냐면 이게 상반기용이 하나 있었고, 하반기용이 하나 있었거든요.'},
   {'time': '6m 4.0s ~ 6m 33.1s', 'student_text': '어떤 걸 보여줄지 모르겠어가지고..'},
   {'time': '6m 4.0s ~ 6m 33.1s', 'student_text': '아니 근데..'},
   {'time': '6m 4.0s ~ 6m 33.1s',
    'student_text': '근데 솔직히 볼 수 있는 게 몇 개 없어가지고..'},
   {'time': '6m 4.0s ~ 6m 33.1s',
    'student_text': '거의.. 아니 그니까 후툰 게 몇 개 있긴 한데요..'},
   {'time': '6m 4.0s ~ 6m 33.1s', 'student_text': '그..'},
   {'time': '6m 4.0s ~ 6m 33.1s', 'student_text': '모르는 게 너무 많아가지고..'}]},
 {'idx': 9,
  'question': '모르는 게 너무 많아가지고..',
  'context': [{'time': '

In [175]:
from prompt import question_classifier

# llm = ChatOpenAI(model='gpt-4o')
llm = ChatOpenAI(
    model='deepseek-chat', 
    openai_api_key=DEEPSEEK_API_KEY, 
    openai_api_base='https://api.deepseek.com',
)

importlib.reload(question_classifier)
system_prompt = question_classifier.QuestionClassifier(subject, user='학생').prompt
prompt = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    ('user', "{user_message}")
])
chain = prompt | llm | StrOutputParser()


question_classifier_results = chain.batch([
    {"user_message": chunk} for chunk in question_context
])

In [179]:
learning_question_context = []
for (res,item) in zip(question_classifier_results, question_context):
    if res == 'True':
        learning_question_context.append(item)
learning_question_context

[{'idx': 9,
  'question': '모르는 게 너무 많아가지고..',
  'context': [{'time': '6m 4.0s ~ 6m 33.1s',
    'student_text': '어떤 걸 보여줄지 모르겠어가지고..'},
   {'time': '6m 4.0s ~ 6m 33.1s', 'student_text': '아니 근데..'},
   {'time': '6m 4.0s ~ 6m 33.1s',
    'student_text': '근데 솔직히 볼 수 있는 게 몇 개 없어가지고..'},
   {'time': '6m 4.0s ~ 6m 33.1s',
    'student_text': '거의.. 아니 그니까 후툰 게 몇 개 있긴 한데요..'},
   {'time': '6m 4.0s ~ 6m 33.1s', 'student_text': '그..'},
   {'time': '6m 4.0s ~ 6m 33.1s', 'student_text': '모르는 게 너무 많아가지고..'},
   {'time': '6m 4.0s ~ 6m 33.1s', 'student_text': '네..'},
   {'time': '6m 4.0s ~ 6m 33.1s', 'student_text': '아니 진짜 충격부모일 수도 있어요.'},
   {'time': '6m 33.2s ~ 6m 33.2s', 'student_text': '네..'},
   {'time': '6m 34.6s ~ 6m 41.5s',
    'teacher_text': '그래서 이 파트가 지금 하반기용이고, 이 도형이 많은 게 상반기용이더라고요.'},
   {'time': '6m 37.4s ~ 7m 1.3s', 'student_text': '그런가요?'}]},
 {'idx': 42,
  'question': '어떤 것부터 시작해야 하는지 모르겠어요.',
  'context': [{'time': '15m 2.6s ~ 15m 2.6s', 'teacher_text': '네.'},
   {'time': '15m 14.3s 

In [180]:
from prompt import student_question_details

llm = ChatOpenAI(
    model='deepseek-chat', 
    openai_api_key=DEEPSEEK_API_KEY, 
    openai_api_base='https://api.deepseek.com',
)

importlib.reload(student_question_details)
system_prompt = student_question_details.QuestionDetails(subject).prompt
prompt = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    ('user', "{user_message}")
])
chain = prompt | llm | StrOutputParser()

# Run it in parallel over chunks
question_details_results = chain.batch([
    {"user_message": chunk} for chunk in learning_question_context
])

In [181]:
result = []
for (res,item) in zip(question_details_results, learning_question_context):
    item['result'] = '학습 질문'
    temp = json.loads(re.search(r'```json\n(.*?)\n```', res, re.DOTALL).group(1))
    if temp['result'] == 'True':
        item['result'] = '구체적인 질문'
        item['reason'] = temp['reason']
    result.append(item)
result

[{'idx': 9,
  'question': '모르는 게 너무 많아가지고..',
  'context': [{'time': '6m 4.0s ~ 6m 33.1s',
    'student_text': '어떤 걸 보여줄지 모르겠어가지고..'},
   {'time': '6m 4.0s ~ 6m 33.1s', 'student_text': '아니 근데..'},
   {'time': '6m 4.0s ~ 6m 33.1s',
    'student_text': '근데 솔직히 볼 수 있는 게 몇 개 없어가지고..'},
   {'time': '6m 4.0s ~ 6m 33.1s',
    'student_text': '거의.. 아니 그니까 후툰 게 몇 개 있긴 한데요..'},
   {'time': '6m 4.0s ~ 6m 33.1s', 'student_text': '그..'},
   {'time': '6m 4.0s ~ 6m 33.1s', 'student_text': '모르는 게 너무 많아가지고..'},
   {'time': '6m 4.0s ~ 6m 33.1s', 'student_text': '네..'},
   {'time': '6m 4.0s ~ 6m 33.1s', 'student_text': '아니 진짜 충격부모일 수도 있어요.'},
   {'time': '6m 33.2s ~ 6m 33.2s', 'student_text': '네..'},
   {'time': '6m 34.6s ~ 6m 41.5s',
    'teacher_text': '그래서 이 파트가 지금 하반기용이고, 이 도형이 많은 게 상반기용이더라고요.'},
   {'time': '6m 37.4s ~ 7m 1.3s', 'student_text': '그런가요?'}],
  'result': '학습 질문'},
 {'idx': 42,
  'question': '어떤 것부터 시작해야 하는지 모르겠어요.',
  'context': [{'time': '15m 2.6s ~ 15m 2.6s', 'teacher_text': '네.'},
  

In [182]:
i = 0
for tt in result:
      if tt['result'] == '구체적인 질문':
            i+=1
            print(tt['question'])
            print(tt['reason'])
            print('---------------------------------')
len(question_context), len(learning_question_context), i

그 단원별로 있는데 거기서 그 범위에서 어떤 것부터 먼저 시작을 해야 되고 어떻게 공부해야 되는지 잘 모르겠어서 네 네
학생이 단원별로 공부할 때 어떤 순서로 시작해야 하는지, 그리고 어떻게 공부해야 하는지에 대해 구체적으로 질문하고 있습니다. 이는 학생이 학습 과정에서 막히거나 이해하지 못한 부분을 명확히 설명하고 있으며, 학습 내용의 맥락에 구체적으로 연결되어 있습니다.
---------------------------------
네네 1차 방정식까지는 괜찮은데 그 그래프 그리고 정비를 반비례 나오고 1차 함수에서 막 직선의 방정식?
학생이 1차 방정식까지는 이해하지만, 그래프 그리기와 직선의 방정식에서 어려움을 겪고 있다고 구체적으로 설명하고 있습니다. 또한, 기울기와 Y절편, X절편을 찾아 그래프를 그리는 데 어려움을 겪고 있다고 명확히 언급하며, 이는 학습 과정에서 막히는 부분을 구체적으로 설명한 것으로 볼 수 있습니다.
---------------------------------
그 닮음을 어떤거랑 어떤거랑 했을땐 되고 어떤거랑 어떤거랑 했을땐 안되는데 뭐랑 했을땐 되는지 모르겠어요.
학생이 닮음의 개념을 이해하는 과정에서 어떤 경우에는 닮음이 성립하고 어떤 경우에는 성립하지 않는지 구체적으로 질문하고 있습니다. 이는 학생이 학습 내용의 맥락에 구체적으로 연결된 질문으로, 구체적인 학습 질문에 해당합니다.
---------------------------------
자 x 아 그 여기를 5x로?
학생이 닮음비를 이용하여 AD와 CD의 길이를 설정하는 방법에 대해 구체적으로 질문하고 있습니다. 이는 학습 내용의 맥락에 구체적으로 연결되어 있으며, 학생이 알고 있는 정보와 새로 배운 정보의 차이를 비교하거나 확인하려는 의도를 포함하고 있습니다.
---------------------------------
여기서 어떻게 해야 하는지 모르겠어요.
학생이 문제 해결 과정에서 막힌 부분을 구체적으로 설명하고 있습니다. 특히, '3x4x를 이렇게 표시하고 ac를

(63, 44, 13)

In [209]:
with open(f"학생_{room_id}.txt", "w", encoding="utf-8-sig") as file:
    json.dump(result, file, ensure_ascii=False, indent=4)

In [173]:
# results = [json.loads(item.strip('```json').strip('```').strip()) for item in results]

# for (res, item) in zip(results, final_feed):
#     item['result'] = res['result']
#     # item['answer'] = res['answer']
#     item['reason'] = res['reason']
# final_feed

[{'idx': 18,
  'question': '그 혹시 내심의 정의가 뭔지 알고 있어요?',
  'context': [{'time': '1m 58.2s ~ 2m 4.3s',
    'teacher_text': '근데 연주 자식이 못해서 어려운 건 아니고 객관적으로 봐도 어려운 게 꽤 있는 것 같아요.'},
   {'time': '2m 5.8s ~ 2m 34.7s', 'teacher_text': '저도 한 2개 정도는 푸는데 꽤 오래 걸렸어요'},
   {'time': '2m 5.8s ~ 2m 34.7s',
    'teacher_text': '일단은 레벨 테스트 2번 문제 한번 볼게요 상반기 2번 문제요'},
   {'time': '2m 5.8s ~ 2m 34.7s',
    'teacher_text': '이게 지금 내심의 정의에 대해서 묻고 있는 거잖아요'},
   {'time': '2m 5.8s ~ 2m 34.7s', 'teacher_text': '점 i는 abc의 내심이다'},
   {'time': '2m 5.8s ~ 2m 34.7s', 'teacher_text': '그 혹시 내심의 정의가 뭔지 알고 있어요?'},
   {'time': '2m 22.6s ~ 2m 27.7s', 'student_text': '네.'},
   {'time': '2m 34.6s ~ 2m 34.9s', 'student_text': '네.'},
   {'time': '2m 35.0s ~ 2m 39.8s', 'student_text': '내 거기 이등분쌈.'},
   {'time': '2m 38.3s ~ 3m 19.5s', 'teacher_text': '그것도 맞는데 이게 정의가 있고 연준학생?'},
   {'time': '2m 39.9s ~ 2m 41.2s', 'student_text': '맞나 이게?'}],
  'result': '이해점검형 질문',
  'reason': "질문은 학생이 내심의 정의를 알고 있는지 확인하기 위한 것으로, '네' 또는 '아니오'로 대답할 수 있는

In [174]:
with open(f"{room_id}.txt", "w", encoding="utf-8-sig") as file:
    json.dump(final_feed, file, ensure_ascii=False, indent=4)